In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelBinarizer
import os

# go to project root dir
os.chdir("../..")

Use spotify feature vector for each track to predict the term:

In [9]:
split = pd.read_csv("tmp/track_train_test_split.csv")
data = pd.read_csv("tmp/track_features.csv")
features = ["danceability", "energy", "valence", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness"]

train_X = data.merge(split).query("is_train").set_index("id")[features]
train_Y = data.merge(split).query("is_train").term
train_Y = LabelBinarizer().fit_transform(train_Y)

test_X = data.merge(split).query("not is_train").set_index("id")[features]
test_Y = data.merge(split).query("not is_train").term
test_Y = LabelBinarizer().fit_transform(test_Y)


print(f"{len(features)} features and {len(train_Y)} samples and {train_Y.shape[1]} classes")

8 features and 480 samples and 12 classes


In [15]:
from keras.layers import Input, Dense

model = keras.Sequential(
    [
        Input(shape=(len(features),)),
        Dense(5, activation="relu"),
        Dense(train_Y.shape[1], activation="softmax"),
    ]
)
model.build()

model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.Accuracy()]
)

model.summary()
model.fit(train_X, train_Y)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 45        
                                                                 
 dense_7 (Dense)             (None, 12)                72        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________
15/15 [==============================] - 0s 1ms/step - loss: 3.0382 - categorical_accuracy: 0.0667 - accuracy: 0.0000e+00


In [21]:
model.evaluate(test_X, test_Y)

4/4 [==============================] - 0s 2ms/step - loss: 2.7013 - categorical_accuracy: 0.0167 - accuracy: 0.0000e+00


[2.7013285160064697, 0.01666666753590107, 0.0]